In [14]:
pip install --upgrade cptac

     |████████████████████████████████| 4.9MB 986kB/s eta 0:00:01
  Found existing installation: cptac 0.6.4
    Uninstalling cptac-0.6.4:
      Successfully uninstalled cptac-0.6.4
Note: you may need to restart the kernel to use updated packages.


In [169]:
import pandas as pd
import numpy as np
import re
import sys
import operator
import collections
import os
import cptac
from datetime import date
#from xml.etree import ElementTree
from xml.dom import minidom

In [207]:
def fixNewLines(s):
    if "\n" in s:
        nList = s.split("\n")
    else:
        nList = []
        nList.append(s)
    return nList

In [208]:
def fixTabs(s):
    if "\t" in s:
        tList = s.split("\t")
    else:
        tList = []
        tList.append(s)
    return tList

In [209]:
def fixParen(s):
    if s.startswith("(") and s.endswith(")"):
        s = s[1:(len(s) - 1)]
    return s

In [216]:
#function to split each item with a newline into two items

#fix if genes are in parenthesis

def fixParsingErrors(interactingList):
    fixedList = []
    for item in interactingList:
        item = item.strip()
        newlineFreeList = fixNewLines(item)
        for n in newlineFreeList:
            tabFreeList = fixTabs(n)
            for t in tabFreeList:
                s = fixParen(t)
                s = s.strip()
                if not s.isspace():
                    fixedList.append(s)
    return list(set(fixedList))

In [217]:
def getUniprotGeneList():
    filePath = '/Users/coribushman/github/cptac/cptac/utils/Uniprot_Proteome.tsv'
    bioplex_interactions = pd.read_csv(filePath, sep='\t')
    uniprotList = []
    for ind, row in bioplex_interactions.iterrows():
        prots = str(row['Gene names'])
        #print(prots)
        prot_arr = prots.split(' ')
        #print(prot_arr)
        for prot in prot_arr:
            if prot not in uniprotList:
                uniprotList.append(prot)
    return uniprotList

In [218]:
def intersectWithUniprot(interactingGenes):
    #get uniprot genes from file
    uniprotGenes = getUniprotGeneList()
    
    #intersect
    geneSet = set(interactingGenes)
    uniprotSet = set(uniprotGenes)   #convert to set
    intersectSet = geneSet.intersection(uniprotSet)
    
    today = date.today()
    todayDate = today.strftime("%m/%d/%y")
    
    print("WikiPathways Interacting Proteins as of " + str(todayDate))
    return intersectSet

In [219]:
def getGenesPerPathwayDict():
    path = '/Users/coribushman/Desktop/wikipathways-20191010-gpml-Homo_sapiens'

    pathwaysPerGene = {}
    genesPerPathway = {}

    for fileName in os.listdir(path):
        if fileName == ".DS_Store":
            continue
        filePath = path + "/" + fileName
        pathwayGenes = []

        xmldoc = minidom.parse(filePath)
        pathway = xmldoc.getElementsByTagName("Pathway")[0]
        pathwayName = pathway.getAttribute("Name")

        dataNodes = pathway.getElementsByTagName("DataNode")
        for node in dataNodes:
            if node.getAttribute("Type") == "GeneProduct":
                geneName = node.getAttribute("TextLabel")
                pathwayGenes.append(geneName)

        genesPerPathway[pathwayName] = pathwayGenes
    return genesPerPathway

In [220]:
def getAllPathways(genesPerPathway):
    allPathways = genesPerPathway.keys()
    allPathways = list(allPathways)
    return allPathways

In [221]:
def getAllGenes(genesPerPathway):
    allGenes = []
    for geneList in genesPerPathway.values():
        allGenes.extend(geneList)

    allGenes = fixParsingErrors(allGenes)
    return allGenes

In [227]:
genesPerPathway = getGenesPerPathwayDict()
allPathways = getAllPathways(genesPerPathway)
allGenes = getAllGenes(genesPerPathway)

#df = pd.DataFrame(genesPerPathway, columns = allPathways, index = allGenes)
df = pd.DataFrame(columns = allPathways, index = allGenes)
df = df.fillna(False)

for column in df.colu

Index(['Target Of Rapamycin (TOR) Signaling', 'NRF2-ARE regulation',
       'PDGF Pathway', 'Eukaryotic Transcription Initiation',
       'Interleukin-11 Signaling Pathway', 'Vitamin D Receptor Pathway',
       'T-Cell antigen Receptor (TCR) pathway during Staphylococcus aureus infection',
       'Phosphatidylcholine catabolism',
       'SRF and miRs in Smooth Muscle Differentiation and Proliferation',
       'Glucocorticoid and Mineralcorticoid Metabolism',
       ...
       'Serotonin Transporter Activity',
       'TCA Cycle (aka Krebs or citric acid cycle)', 'Serotonin and anxiety',
       'Sudden Infant Death Syndrome (SIDS) Susceptibility Pathways',
       'Canonical NF-KB pathway',
       'Photodynamic therapy-induced HIF-1 survival signaling',
       'Arrhythmogenic Right Ventricular Cardiomyopathy',
       'Ethanol metabolism resulting in production of ROS by CYP2E1',
       'Interactions between immune cells and microRNAs in tumor microenvironment',
       'Metabolism of Tetra

['a', '']


OLD FUNCTIONS

In [47]:
wikiPathwaysInterGenes = findInteractions("p53")
filteredInterGenes = fixParsingErrors(wikiPathwaysInterGenes)
finalGeneSet = intersectWithUniprot(filteredInterGenes)
print(finalGeneSet)

WikiPathways Interacting Proteins as of 10/25/19
{'NOXA1', 'SIRT1', 'CHK1', 'PTEN', 'ATR', 'EP300', 'IL2', 'MDM2', 'CCND1', 'CDKN1A', 'HIPK2', 'DCAKD', 'PRKDC', 'SFN', 'PIAS1', 'RELA', 'SIAH1', 'GADD45A', 'HDAC1', 'ATM', 'CHK2', 'CBP', 'BRCA1', 'MDM4', 'CSNK1D', 'IL6', 'PML', 'AURKA', 'FADD', 'PUMA'}


In [39]:
#size=-1 to get all genes, but takes a long time
#if statements might need to be added in case keys not in dict when parsing

#this function will query wikipathways for all interactions of a given gene for the Homo Sapiens species
#some interactions won't be genes

import urllib3
import json

def findInteractions(geneName): #retuns list of interacting elements
    interactionList = []
    
    try:
        requestURL = "http://webservice.wikipathways.org/findInteractions?query=" + geneName + "&format=json" 
        http = urllib3.PoolManager()
        response = http.request('GET', requestURL)

        if response.status == 200: #if success, get data from API response
            responseData = response.data
            #print(type(responseData))
            responseDataDict = json.loads(responseData)
            #print(responseDataDict)
            resultList = list(responseDataDict.values())
            interactionsList = resultList[0]

            for interactionDict in interactionsList:

                if interactionDict["species"] == "Homo sapiens":
                    rightValuesList = interactionDict["fields"]["right"]["values"]
                    for value in rightValuesList:
                        if (value not in interactionList) and (geneName.casefold() not in value.casefold()):
                            interactionList.append(value)

                    leftValuesList = interactionDict["fields"]["left"]["values"]
                    for value in leftValuesList:
                        if (value not in interactionList) and (geneName.casefold() not in value.casefold()):
                            interactionList.append(value)
                else:
                    continue       
        else:
            print("Something went wrong with the response status")

        return interactionList
    except:
        print("There was a problem getting a response from the WikiPathways API")

In [94]:
#NEW FUNCTION WITH XML FILES

def parseWikipathways():
    pathwaysPerGene = {}
    genesPerPathway = {}
    
    path = '/Users/coribushman/Desktop/wikipathways_homo_sapiens'
    for fileName in os.listdir(path):
        filePath = path + "/" + fileName
        
        dom = ElementTree.parse(filePath)
        pathwayName = dom.findall("Pathway/Name")
        print(pathwayName)
        
        with open(filePath, 'r') as file:
            fileContents = file.read()
            fileContents = fileContents.rstrip("\n")
            pattern = "Name=*\n* Version="
            print(fileContents)
            if re.search(pattern, fileContents, re.MULTILINE):
                
                pathwayName = re.findall(r"Name=*\n* Version=", fileContents, re.MULTILINE)
                print(pathwayName)
                print("done")

parseWikipathways()

[]
<?xml version="1.0" encoding="UTF-8"?>
<Pathway xmlns="http://pathvisio.org/GPML/2013a" Name="4-hydroxytamoxifen, Dexamethasone, and Retinoic Acids Regulation of p27 Expression" Version="20160816" Organism="Homo sapiens">
  <Comment Source="WikiPathways-description">This pathway is modeled after Figure 7 in the article ''Upstream molecular signaling pathways of p27(Kip1) expression: effects of 4-hydroxytamoxifen, dexamethasone, and retinoic acids'' (See Bibliography). Several anticancer agents up-regulate expression of p27, a cyclin-dpendent kinase inhibitor that blocks the transition from G1 to S phase. The pathways used by the anticancer agents (retinoic acids, 4-hydroxytamoxifen, dexamethasone) are shown above from human breast cancer cells.</Comment>
  <BiopaxRef>f5c</BiopaxRef>
  <Graphics BoardWidth="1659.0" BoardHeight="820.0"/>
  <DataNode TextLabel="Triciribine" GraphId="a188c" Type="Metabolite">
    <Graphics CenterX="311.0" CenterY="262.0" Width="80.0" Height="20.0" ZOrde

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [7]:
#METHOD 2: GO THROUGH EACH GENE IN MY UNFILTERED LIST AND SEE IF IT IS IN UNIPROT
        
import certifi
import json
import urllib3
#import requests, sys

def geneInUniprot(geneName):  #pass in the interacting list from findInteractions
    try:
        requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&gene=" + geneName + "&organism=homo%20sapiens&format=json"
        http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
        response = http.request('GET', requestURL)
    except:
        return "n"

    if response.status == 200: #if success, get data from API response
        responseData = response.data
        responseDataList = json.loads(responseData)
        
        
        
        if (len(responseDataList) != 0) and ("accession" in responseDataList[0].keys()) and (geneName != "DNA"):
            return geneName
        else:
            return "n"
    else:
        #print("Something went wrong with the response status")
        return "n"
        
#p = geneInUniprot("TP53")
#print(p)

In [8]:
#NEW
#REMOVE DNA LATER

import certifi
import json
import urllib3
#import requests, sys

def geneInUniprotNew(geneList):  #pass in the interacting list from findInteractions
    try:
        requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&exact_gene=" + geneList + "&organism=homo%20sapiens&format=json"
        http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
        response = http.request('GET', requestURL)
    except:
        return ["API request didn't work"]

    if response.status == 200: #if success, get data from API response
        responseData = response.data
        responseDataList = json.loads(responseData)
        
        filteredList = []
        for data in responseDataList:
            filteredList.append(data["gene"][0]["name"]["value"])
            #if "accession" in data.keys():
                #filteredList.append(data["gene"]["name"]["value"])
                
        filteredSet = set(filteredList)
        #print("geneList: " + geneList)
        #print("filteredSet: " + str(filteredSet))
        return filteredSet
        
    else:
        #print("Something went wrong with the response status")
        return ["response status is not 200"]

In [9]:
import time
start_time = time.time()

interactingList = findInteractions("TP53")

fixedInteractingList = fixNewlines(interactingList)
#print(sorted(fixedInteractingList))
#print("----------------------------------------------------------------")

goodList = []    
listForFunct = []
print(fixedInteractingList)

for gene in fixedInteractingList:
    listForFunct.append(gene)
    if len(listForFunct) == 20:
        inputStr = ",".join(listForFunct)
        print("inputStr: " + inputStr)
        goodList += (geneInUniprotNew(inputStr))
        listForFunct = []

if len(listForFunct) > 0:
    inputStr = ",".join(listForFunct)
    goodList += (geneInUniprotNew(inputStr))

print(goodList)

print("--- %s seconds ---" % (time.time() - start_time))

['Ubiquitin-mediated', 'degradation', 'SNAI2', 'Calcium', 'Substrate &eacute;', 'ATM', 'ATR', 'BLM', 'CDC25C', 'MDM2', 'del17p', '', 'MIR15A', 'MIR16-1', 'Chk2', 'DNA', 'CREBBP', 'ARF', '(CDKN2A)', 'MTA2', 'Accumulation DNA damage', 'SREBF1', 'Nuclear abnormalities', 'RCHY1', 'DNA damage', 'DNA Damage', 'MYC', 'HMGB1', 'APC', 'TGFB', 'OTUD5', 'RFC4', 'CTNNB1', 'SNURF', 'PLAC8', 'CHEK2', 'Hypoxia', 'nitric oxide', 'TP73 Delta Np73', 'SUMO-1', 'OTX2', 'TNFSF10', 'MAPK14', 'MAPK13', 'MAPK12', 'MAPK11', 'Chk1', 'UCHL1', 'MEG3', 'P85A_HUMAN', 'CDC42', 'CAPN1', 'CAPN2', 'MIR1285-2', 'GRIN1', 'RHOC', 'RHOA', 'ROCK2', 'RHOB', 'cenersen sodium', 'PRKAB1', 'PRKAG1', 'PRKAA1', 'IFI16', 'GSK3B', 'ING1', 'BCL2', 'ROS', 'EP300', 'MDM4', 'p14 ARF ', '(CDKN2A)', 'HPV E6', 'PCNA', 'MIR34B', 'MIR34A', 'MIR34C', 'Senescence', 'p21', 'Apoptosis', 'Cyclin B', 'CDK1', 'GADD45A', 'p53 pathway', 'BAX', 'CDKN1A', 'GADD45G', 'BAK1', 'POLK', 'DDB2', 'GADD45B', 'uc339', '(Entraper1)', 'NANOG', 'hsa-mir-143/145 cl

In [110]:
#Method 2: check individually if each gene in interacting list is a uniprot gene
#FASTER

import time
start_time = time.time()

interactingList = findInteractions("TP53")

#filteredGenes = [geneInUniprot(x) for x in interactingList if geneInUniprot(x) != "n"]

fixedInteractingList = fixNewlines(interactingList)

filteredGenes = []
for gene in fixedInteractingList:
    if geneInUniprot(gene) != "n":
        filteredGenes.append(gene)
print(len(filteredGenes))
print(filteredGenes)

print("--- %s seconds ---" % ((time.time() - start_time)))

123
['SNAI2', 'ATM', 'ATR', 'BLM', 'CDC25C', 'MDM2', 'Chk2', 'CREBBP', 'ARF', 'MTA2', 'SREBF1', 'RCHY1', 'MYC', 'HMGB1', 'APC', 'TGFB', 'OTUD5', 'RFC4', 'CTNNB1', 'SNURF', 'PLAC8', 'CHEK2', 'OTX2', 'TNFSF10', 'MAPK14', 'MAPK13', 'MAPK12', 'MAPK11', 'Chk1', 'UCHL1', 'CDC42', 'CAPN1', 'CAPN2', 'GRIN1', 'RHOC', 'RHOA', 'ROCK2', 'RHOB', 'PRKAB1', 'PRKAG1', 'PRKAA1', 'IFI16', 'GSK3B', 'ING1', 'BCL2', 'ROS', 'EP300', 'MDM4', 'PCNA', 'CDK1', 'GADD45A', 'BAX', 'CDKN1A', 'GADD45G', 'BAK1', 'POLK', 'DDB2', 'GADD45B', 'NANOG', 'PMAIP1', 'BBC3', 'BOK', 'THBS1', 'BCL2L11', 'CDKN1C', 'CDKN1B', 'ATAD2', 'RAD17', 'UBE2T', 'COL9A3', 'COL9A1', 'MTDH', 'LBR', 'EGFR', 'RFC3', 'UBE2C', 'FANCI', 'CEBPZ', 'RNF144B', 'S100A6', 'PIDD', 'CDK5R1', 'SLC11A2', 'ING2', 'SERPINE1', 'SESN1', 'RRM2B', 'FAS', 'TNFRSF10B', 'CCNB3', 'CCNB2', 'CCNB1', 'SFN', 'SAT2', 'SAT1', 'CASP8', 'SESN2', 'DDIT4', 'CDC2', 'ATF2', 'ELK1', 'p38', 'MAPK8', 'JNK', 'MAPK9', 'MAPK10', 'BMF', 'BID', 'BNIP3L', 'BNIP3', 'HIF1A', 'ARNT', 'RB1', 